In [1]:
#all the needed Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from bs4 import BeautifulSoup

In [2]:
df = pd.read_excel("sys_id.xlsx")

In [3]:
df

,Number,Short description,Author,Article body
0,KB0017514,Update Password Action in Flow Designer,Georgios Nikas,"<p class=""cm-comment-text-detail ng-binding ng..."
1,KB0017513,Service Portal form cell spacing or padding no...,Georgios Nikas,"<p class=""cm-comment-text-detail ng-binding ng..."
2,KB0017512,Reporting on flagged field in knowledge manage...,Georgios Nikas,"<p class=""cm-comment-text-detail ng-binding ng..."
3,KB0017511,Ticket Conversation Widget in Service Portal -...,Georgios Nikas,"<p class=""cm-comment-text-detail ng-binding ng..."
4,KB0017510,Service portal basics - how do I print data fr...,Georgios Nikas,"<p class=""cm-comment-text-detail ng-binding ng..."
...,...,...,...,...
5750,KB0011761,buttons on email notification,Georgios Nikas,"<p class=""cm-comment-text-detail ng-binding ng..."
5751,KB0011760,Create object from XML attachment larger than 5MB,Georgios Nikas,"<p class=""cm-comment-text-detail ng-binding ng..."
5752,KB0011759,addQuery and addEncodedQuery queries not worki...,Georgios Nikas,"<p class=""cm-comment-text-detail ng-binding ng..."
5753,KB0011758,Query to find max date,Georgios Nikas,"<p class=""cm-comment-text-detail ng-binding ng..."


In [4]:
for column in df.columns[3:]:
    for index in range(len(df[column])):
        #slicing the column
        soup = BeautifulSoup(""+df.at[index,'Article body']+"")
        soup_ = soup.get_text()
        soup_ = soup_.replace("\n", "")
        soup_ = soup_.replace("\xa0", "")
        soup_ = soup_.replace("\\" ,  "" )
        df.at[index,'Article body'] = soup_
df = df.astype({"Short description": str})
index_list = []
for index, row in df.iterrows():
        if((len(row["Short description"]) < 15) | (len(row["Short description"])> 861)):
            df.drop(index, inplace=True)

In [5]:
def clean_sentence(s, stopWords=False):
    s = s.lower().strip()
    #s = re.sub(r'[^a-z0-9\s]',' ',s)
    if stopWords:
        s = remove_stopwords(s)
    return s

def get_cleaned(df, stopWords=False):
    sents = df[["Short description"]]
    cleaned_sentences = []
    for index, row in df.iterrows():
        cleaned = clean_sentence(row["Short description"], stopWords)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences
cleaned_s_with_not_stopword = get_cleaned(df, stopWords=True)

In [6]:
from bert_serving.client import BertClient
bc = BertClient()

In [7]:
#run this cell only once !!!!!!!!!!!!!!!!!!!!!!
questions_embedding = []
for question_ in cleaned_s_with_not_stopword:
    questions_embedding.append(bc.encode([question_]))

In [42]:
def retrieveAndPrintAnswer(question_embedding, sentence_embeddings, df, sentences):
    max_sim = -1
    index_sim = -1
    potential_questions_indexes = []
    potential_questions = []
    answers = {}
    for index, faq_embedding in enumerate(sentence_embeddings):
        sim = cosine_similarity(faq_embedding, question_embedding)[0][0]    
        if sim>max_sim:
            max_sim = sim
            index_sim = index
            if(index_sim!=0):
                potential_questions_indexes.append(df.iloc[index_sim,0])
                potential_questions.append(df.iloc[index_sim,3])
    if(len(potential_questions)>11):
        potential_questions_indexes = potential_questions_indexes[:10]
        potential_questions= potential_questions[:10]
    answers = dict( zip(potential_questions_indexes,potential_questions))
    return answers

In [43]:
def getAnswers(question):
    question = clean_sentence(question, stopWords=True)
    question_embedding = bc.encode([question])
    answers = retrieveAndPrintAnswer(question_embedding, questions_embedding, df, cleaned_s_with_not_stopword)
    return answers   

In [48]:
df["Short description"][2]

'Reporting on flagged field in knowledge management'

In [53]:
#test
answers = getAnswers("How to report on flagged field in knowledge management?")

In [54]:
import json
print(json.dumps(answers, sort_keys=False, indent=4))

{
    "KB0017513": "I found out this is a problem with ServiceNow. I have problem ticket open with ServiceNow HI Support, so we will see if they resolve the issue. In the mean as a solution, I add 8px white border around the image so it achieved the sameeffect.-Wesley",
    "KB0017512": "This would give you the date the article was flagged. It creates a new entry when articles are flagged, rated, commented, etc.No scripting needed.  Schedule that and check the box that says \"Don't send if empty\" (or something like that). I think that meets your requirements.",
    "KB0017415": "Hi Ronald,Having a field in multiple places in a form doesn't sounds right. And having 150+ fields within a form sounds like asking for trouble.It sounds like you rather should have used ACLs (security rules) or UI policies or even Views so that the users see the fields they only need to see.Thanks,Berny",
    "KB0015401": "Hi Varun,Have you tried list editing? You can click-select (drag) to highlight and chan